In [1]:
import tensorflow as tf
import tensornetwork as tn
import numpy as np
import tensornetwork as tn
class TNLayer(tf.keras.layers.Layer):
 
  def __init__(self,Tensor_dimention = 20):
    super(TNLayer, self).__init__()
    # Create the variables for the layer.
    

    self.A1 = tf.Variable(tf.random.normal(shape=[4,Tensor_dimention,8], stddev=1.0/256.0),name="a1", trainable=True)
    self.A2 = tf.Variable(tf.random.normal(shape=[4,Tensor_dimention,Tensor_dimention,4], stddev=1.0/256.0),name="a2", trainable=True)
    self.A3 = tf.Variable(tf.random.normal(shape=[4,Tensor_dimention,Tensor_dimention,4], stddev=1.0/256.0),name="a3", trainable=True)
    self.A4 = tf.Variable(tf.random.normal(shape=[4,Tensor_dimention,12], stddev=1.0/256.0),name="a4", trainable=True)

    self.bias = tf.Variable(tf.zeros(shape=(256*6)), name="bias", trainable=True)
 
  def call(self, inputs):
    # Define the contraction.
    # We break it out so we can parallelize a batch using
    # tf.vectorized_map (see below).
    Nodes = [tn.Node(self.A1,'a0',backend="tensorflow")]
    Nodes+=[tn.Node(self.A2,f'a{1}',backend="tensorflow")]
    Nodes+=[tn.Node(self.A3,f'a{2}',backend="tensorflow")]
    Nodes+=[tn.Node(self.A4,f'a{3}',backend="tensorflow")]
    def f(input_vec, Nodes, bias_var):
      # Reshape to a matrix instead of a vector.
      input_vec = tf.reshape(input_vec, [200,4,4,4,4])
      T_node = tn.Node(input_vec, backend="tensorflow",name = 't')
      for i in range(len(Nodes)-1):
          if i == 0:
              Nodes[i][1]^Nodes[i+1][1]
          else:
              Nodes[i][2]^Nodes[i+1][1]

      for i in range(len(Nodes)):
          if i == 0 or i == (len(Nodes))-1:
              Nodes[i][0]^T_node[i+1]
          else:
              Nodes[i][3]^T_node[i+1]
      A = tn.to_graphviz(Nodes+[T_node])     
      contraction = T_node@Nodes[0]
      for i in range(1,len(Nodes)):
        contraction = contraction@Nodes[i]
      result = tf.reshape(contraction.tensor,[200,256*6])

      # To make the code shorter, we also could've used Ncon.
      # The above few lines of code is the same as this:
      # result = tn.ncon([x, a_var, b_var], [[1, 2], [-1, 1, 3], [-2, 2, 3]])
 
      # Finally, add bias.
      return result + bias_var
  
    # To deal with a batch of items, we can use the tf.vectorized_map
    # function.
    # https://www.tensorflow.org/api_docs/python/tf/vectorized_map
    result = tf.vectorized_map(
        lambda vec: f(vec, Nodes, self.bias), inputs)
    return result

2023-12-21 11:38:27.940582: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-21 11:38:28.399206: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
temp = TNLayer()
temp(tf.random.normal([2,200,256]))

2023-12-21 11:38:28.999731: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-21 11:38:29.017150: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-21 11:38:29.017288: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

<tf.Tensor: shape=(2, 200, 1536), dtype=float32, numpy=
array([[[-3.65314662e-07, -1.49800996e-07,  5.95339316e-07, ...,
         -1.28110244e-07,  2.79519895e-07,  4.38398331e-07],
        [ 2.65480196e-07,  1.62413130e-07, -8.12657674e-07, ...,
          3.55212592e-07,  4.50027244e-07, -7.89342067e-08],
        [ 3.94209508e-07,  1.02907200e-06,  4.47108192e-07, ...,
         -1.26556301e-07, -3.06222375e-07, -4.84874704e-07],
        ...,
        [-5.50077004e-07, -4.33651678e-07,  1.42126709e-07, ...,
         -1.87102785e-07,  1.48502437e-08, -5.01837690e-07],
        [-6.09619178e-07, -9.85340193e-07, -1.16776243e-06, ...,
         -3.01511164e-07, -4.48813580e-07,  5.16206171e-08],
        [ 5.77559831e-07, -6.32321303e-07,  8.43774160e-07, ...,
          5.54238227e-07, -1.59658399e-07,  4.03335108e-08]],

       [[-3.90378830e-07, -1.06017062e-07, -4.90865318e-07, ...,
          1.91617460e-07,  2.38905109e-08,  3.00571486e-07],
        [ 5.14403609e-07, -7.55888223e-08, -2.7

In [3]:
import tensorflow as tf

class MultiheadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads,classical_rate = 0.25,Tensor_dimention = 20):
        super(MultiheadAttention, self).__init__()
        assert d_model % num_heads == 0
        self.num_heads = num_heads
        self.depth = d_model
        self.rate = classical_rate
        assert int(num_heads/classical_rate) == num_heads/classical_rate



        self.wq = tf.keras.layers.Dense(int(d_model)*int(num_heads*classical_rate))
        self.wk = tf.keras.layers.Dense(int(d_model)*int(num_heads*classical_rate))
        self.wv = tf.keras.layers.Dense(int(d_model)*int(num_heads*classical_rate))

        self.wq_tensor = TNLayer(Tensor_dimention)
        self.wk_tensor = TNLayer(Tensor_dimention)
        self.wv_tensor = TNLayer(Tensor_dimention)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, int(self.num_heads*self.rate), self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    def split_heads_tensor(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, int(self.num_heads*(1-self.rate)), self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    def call(self, q, k, v, mask):
        batch_size = tf.shape(q)[0]
        
        print(q.shape)
        q_class = self.split_heads(self.wq(q), batch_size)
        k_class = self.split_heads(self.wk(k), batch_size)
        v_class = self.split_heads(self.wv(v), batch_size)
        print(q_class.shape)
        q_tensor = self.split_heads_tensor(self.wq_tensor(q),batch_size)
        k_tensor = self.split_heads_tensor(self.wq_tensor(k),batch_size)
        v_tensor = self.split_heads_tensor(self.wq_tensor(v),batch_size)
        print(q_tensor.shape)
        q,k,v = tf.concat([q_class,q_tensor],axis=1),tf.concat([k_class,k_tensor],axis=1),tf.concat([v_class,v_tensor],axis=1)

        scaled_attention, attention_weights = self.scaled_dot_product_attention(q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.depth*self.num_heads))

        output = self.dense(concat_attention)
        return output, attention_weights

    def scaled_dot_product_attention(self, q, k, v, mask):
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

        if mask is not None:
            scaled_attention_logits += (mask * -1e9)

        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, v)
        return output, attention_weights

In [4]:
input = tf.keras.layers.Input([200,256])
output,_ = MultiheadAttention(256,8)(input,input,input,None)
output = output
model = tf.keras.Model(inputs  = [input], outputs = [output])
model.summary()


(None, 200, 256)
(None, 2, None, 256)
(None, 6, None, 256)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 200, 256)]   0           []                               
                                                                                                  
 multihead_attention (Multihead  ((None, None, 256),  967104     ['input_1[0][0]',                
 Attention)                      (None, 8, None, No               'input_1[0][0]',                
                                ne))                              'input_1[0][0]']                
                                                                                                  
Total params: 967,104
Trainable params: 967,104
Non-trainable params: 0
______________________________________________________________

In [7]:
model(tf.random.normal([256,200,256]))

(256, 200, 256)
(256, 2, 200, 256)
(256, 6, 200, 256)


<tf.Tensor: shape=(256, 200, 256), dtype=float32, numpy=
array([[[-3.73702450e-03, -2.06267834e-02, -6.05356656e-02, ...,
          4.13719714e-02, -3.59384567e-02,  1.32235205e-02],
        [-1.19590359e-02, -3.89066688e-03, -3.07100173e-02, ...,
          7.09708631e-02, -4.08575051e-02,  4.52038087e-02],
        [-3.86949293e-02, -5.45938537e-02, -1.76048763e-02, ...,
          8.40343069e-03,  9.62884910e-03,  3.09624784e-02],
        ...,
        [-4.61964235e-02, -4.60500792e-02,  2.37633586e-02, ...,
          8.43577459e-02, -1.05146300e-02,  4.80099656e-02],
        [-1.54572548e-02, -6.22609109e-02, -2.05411986e-02, ...,
          2.13305112e-02, -2.25147698e-02,  4.18667644e-02],
        [ 3.44831198e-02, -6.64576516e-02, -1.21740317e-02, ...,
          2.87348609e-02, -8.75886306e-02,  3.03424243e-02]],

       [[-3.27780060e-02,  6.54790029e-02, -2.71975864e-02, ...,
          1.53236426e-02,  3.67707200e-02, -4.15575178e-03],
        [ 5.81410117e-02,  1.28691187e-02,  3.

In [12]:
import graphviz

In [182]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim
        self.query_dense = tf.keras.layers.Dense(embedding_dim*num_heads)
        self.key_dense = tf.keras.layers.Dense(embedding_dim*num_heads)
        self.value_dense = tf.keras.layers.Dense(embedding_dim*num_heads)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)
        print(query)
        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        print(scaled_attention.shape)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim*num_heads))
        outputs = self.dense(concat_attention)
        return outputs

In [183]:
input = tf.keras.layers.Input([200,256],batch_size=32)
output = MultiHeadAttention(256,8)(input)
model = tf.keras.Model(inputs  = [input], outputs = [output])

Tensor("multi_head_attention_60/transpose:0", shape=(32, 8, 200, 256), dtype=float32)
(32, 8, 200, 256)


In [184]:
model.summary()

Model: "model_56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_72 (InputLayer)       [(32, 200, 256)]          0         
                                                                 
 multi_head_attention_60 (Mu  (32, 200, 256)           2103552   
 ltiHeadAttention)                                               
                                                                 
Total params: 2,103,552
Trainable params: 2,103,552
Non-trainable params: 0
_________________________________________________________________


In [148]:
input = tf.keras.layers.Input([200,256],batch_size=32)
output,_ =  MultiheadAttention(256,16)(input,input,input,None)

model = tf.keras.Model(inputs  = [input], outputs = [output])

(32, 200, 256)
(32, 4, 200, 16)


In [150]:
output

<KerasTensor: shape=(32, 200, 256) dtype=float32 (created by layer 'multihead_attention_11')>

In [149]:
model.summary()

Model: "model_47"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_60 (InputLayer)          [(32, 200, 256)]     0           []                               
                                                                                                  
 multihead_attention_11 (Multih  ((32, 200, 256),    155792      ['input_60[0][0]',               
 eadAttention)                   (32, 16, 200, 200)               'input_60[0][0]',               
                                )                                 'input_60[0][0]']               
                                                                                                  
Total params: 155,792
Trainable params: 155,792
Non-trainable params: 0
__________________________________________________________________________________________________


In [32]:
a(tf.random.normal([1,200,256]))

<tf.Tensor: shape=(1, 200, 256), dtype=float32, numpy=
array([[[-0.7576784 ,  0.5117144 ,  1.3913641 , ...,  1.569271  ,
         -1.1123393 ,  0.88063514],
        [-0.4770038 ,  0.149418  , -1.6568892 , ...,  0.24261492,
         -0.3732225 , -0.92699033],
        [-0.93850803,  0.9730366 ,  0.20917985, ...,  0.29123157,
         -0.25967714,  1.2175975 ],
        ...,
        [ 1.7240776 , -0.39755765,  1.0520613 , ...,  0.11004458,
          1.0479846 ,  0.4702791 ],
        [-0.31224385, -0.1991196 , -0.25010037, ..., -0.78511393,
          0.2746125 ,  0.08908027],
        [ 0.37627482,  0.6397044 ,  0.39135456, ...,  0.52852196,
         -0.5584524 ,  0.23717976]]], dtype=float32)>

In [13]:
TN = TNLayer()
A = np.random.random([200,256])

In [6]:
TN(np.array([A,A,A,A,A]))

2023-12-20 11:49:18.616703: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


<tf.Tensor: shape=(5, 200, 192), dtype=float32, numpy=
array([[[ 3.9458379e-07, -3.0122372e-08, -4.7138479e-07, ...,
          1.6917920e-06, -2.0015385e-07, -4.3961748e-07],
        [ 1.5410170e-07, -3.6303497e-08, -3.7850884e-07, ...,
          8.8086756e-07,  3.2530153e-07,  2.0509961e-07],
        [-3.6292690e-07,  3.0433881e-07, -7.1985124e-07, ...,
          1.1278707e-06,  1.3492263e-07,  2.0843441e-07],
        ...,
        [ 1.1674189e-08,  1.3615391e-07, -8.3924039e-07, ...,
          8.6666080e-07, -7.7096711e-09, -3.6934164e-08],
        [-2.1719433e-07, -9.0083233e-08, -4.3829752e-07, ...,
          1.1408886e-06,  3.2110330e-09,  1.1046866e-07],
        [-2.3101596e-07, -9.6460667e-08, -5.1828181e-07, ...,
          7.0417246e-07,  2.7571778e-07, -6.1455154e-07]],

       [[ 3.9458379e-07, -3.0122372e-08, -4.7138479e-07, ...,
          1.6917920e-06, -2.0015385e-07, -4.3961748e-07],
        [ 1.5410170e-07, -3.6303497e-08, -3.7850884e-07, ...,
          8.8086756e-07,  3.

In [7]:
MultiheadAttention

# 테스트
d_model = 256
num_heads = 8
input_seq_len = 200
batch_size = 2048

multihead_attention = MultiheadAttention(d_model, num_heads,Tensor_dimention=15)
input_q = tf.random.normal((batch_size, input_seq_len, d_model))
input_k = tf.random.normal((batch_size, input_seq_len, d_model))
input_v = tf.random.normal((batch_size, input_seq_len, d_model))
mask = None  # Optional mask to specify which positions should be masked
output, attention_weights = multihead_attention(input_q, input_k, input_v, mask)
print("Multihead Attention Output shape:", output.shape)
print("Attention Weights shape:", attention_weights.shape)

2023-12-20 11:49:29.730378: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 400.00MiB (rounded to 419430400)requested by op AddV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-12-20 11:49:29.730412: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-12-20 11:49:29.730424: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 4, Chunks in use: 4. 1.0KiB allocated for chunks. 1.0KiB in use in bin. 16B client-requested in use in bin.
2023-12-20 11:49:29.730432: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 6, Chunks in use: 5. 4.5KiB allocated for chunks. 3.8KiB in use in bin. 3.6KiB client-requested in use in bin.
2023-12-20 11:49:29.730439: I tensorflow/tsl/framework/bfc_allocato

RuntimeError: pybind11::error_already_set: MISMATCH of original and normalized active exception types: ORIGINAL _NotOkStatusException REPLACED BY KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /home/yslee/.local/lib/python3.8/site-packages/tensorflow/python/eager/core.py(36): __init__
  /home/yslee/.local/lib/python3.8/site-packages/tensorflow/python/ops/gen_math_ops.py(461): add_v2
  /home/yslee/.local/lib/python3.8/site-packages/tensorflow/python/ops/math_ops.py(4022): add
  /home/yslee/.local/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py(1176): op_dispatch_handler
  /home/yslee/.local/lib/python3.8/site-packages/tensorflow/python/util/traceback_utils.py(150): error_handler
  /home/yslee/.local/lib/python3.8/site-packages/tensorflow/python/ops/random_ops.py(94): random_normal
  /home/yslee/.local/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py(1176): op_dispatch_handler
  /home/yslee/.local/lib/python3.8/site-packages/tensorflow/python/util/traceback_utils.

In [9]:
input_q = tf.random.normal((2, input_seq_len, d_model))
input_q 

<tf.Tensor: shape=(2, 200, 256), dtype=float32, numpy=
array([[[-0.4862139 ,  0.96792114,  0.42404115, ...,  0.09001559,
          0.46600857, -1.0089929 ],
        [-0.62987536, -2.5237682 , -0.35136494, ...,  0.6651539 ,
          0.5311567 , -0.04648928],
        [ 0.3532162 , -0.49647188, -1.1656084 , ...,  0.29334262,
          0.0310472 , -0.9394325 ],
        ...,
        [ 0.0506833 ,  1.1525031 ,  1.6949191 , ...,  0.44479018,
          0.26881033, -0.38094622],
        [-1.8940521 , -0.80030406,  1.1014678 , ..., -0.7414971 ,
          1.4075471 , -2.53017   ],
        [-0.9457627 , -0.8573482 , -0.49087054, ...,  0.01483133,
         -0.2940595 ,  0.8741598 ]],

       [[ 0.03978452,  1.0761245 ,  1.0841494 , ..., -1.2231328 ,
         -0.55342877,  1.5121046 ],
        [-1.4462594 ,  0.3899501 ,  0.06220459, ..., -0.28765523,
         -0.9056579 ,  1.0569195 ],
        [-0.14841177,  0.57795256,  0.14500858, ..., -1.0352979 ,
         -0.8812449 , -0.83084583],
        ...,

In [46]:
multihead_attention.count_params()

138572

In [44]:
import tensorflow as tf

class MultiheadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiheadAttention, self).__init__()
        assert d_model % num_heads == 0
        self.num_heads = num_heads
        self.depth = d_model // num_heads
        
        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, q, k, v, mask):
        batch_size = tf.shape(q)[0]

        q = self.split_heads(self.wq(q), batch_size)
        k = self.split_heads(self.wk(k), batch_size)
        v = self.split_heads(self.wv(v), batch_size)
        print(q.shape)

        scaled_attention, attention_weights = self.scaled_dot_product_attention(q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, d_model))

        output = self.dense(concat_attention)
        return output, attention_weights

    def scaled_dot_product_attention(self, q, k, v, mask):
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

        if mask is not None:
            scaled_attention_logits += (mask * -1e9)

        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, v)
        return output, attention_weights

In [42]:
import tensorflow as tf

class MultiheadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiheadAttention, self).__init__()
        assert d_model % num_heads == 0
        self.num_heads = num_heads
        self.depth = d_model // num_heads
        
        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, q, k, v, mask):
        batch_size = tf.shape(q)[0]

        q = self.split_heads(self.wq(q), batch_size)
        k = self.split_heads(self.wk(k), batch_size)
        v = self.split_heads(self.wv(v), batch_size)
        print(q.shape)

        scaled_attention, attention_weights = self.scaled_dot_product_attention(q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, d_model))

        output = self.dense(concat_attention)
        return output, attention_weights

    def scaled_dot_product_attention(self, q, k, v, mask):
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

        if mask is not None:
            scaled_attention_logits += (mask * -1e9)

        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, v)
        return output, attention_weights

# 테스트
d_model = 256
num_heads = 8
input_seq_len = 200
batch_size = 2048

multihead_attention = MultiheadAttention(d_model, num_heads)
input_q = tf.random.normal((batch_size, input_seq_len, d_model))
input_k = tf.random.normal((batch_size, input_seq_len, d_model))
input_v = tf.random.normal((batch_size, input_seq_len, d_model))
mask = None  # Optional mask to specify which positions should be masked

output, attention_weights = multihead_attention(input_q, input_k, input_v, mask)
print("Multihead Attention Output shape:", output.shape)
print("Attention Weights shape:", attention_weights.shape)

2023-12-20 16:46:46.669389: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 400.00MiB (rounded to 419430400)requested by op RandomStandardNormal
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-12-20 16:46:46.669417: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-12-20 16:46:46.669426: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 23, Chunks in use: 23. 5.8KiB allocated for chunks. 5.8KiB in use in bin. 96B client-requested in use in bin.
2023-12-20 16:46:46.669431: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 5, Chunks in use: 4. 3.8KiB allocated for chunks. 3.0KiB in use in bin. 2.9KiB client-requested in use in bin.
2023-12-20 16:46:46.669436: I tensorflow/tsl/frame

RuntimeError: pybind11::error_already_set: MISMATCH of original and normalized active exception types: ORIGINAL _NotOkStatusException REPLACED BY KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /home/yslee/.local/lib/python3.8/site-packages/tensorflow/python/eager/core.py(36): __init__
  /home/yslee/.local/lib/python3.8/site-packages/tensorflow/python/ops/gen_random_ops.py(631): random_standard_normal
  /home/yslee/.local/lib/python3.8/site-packages/tensorflow/python/ops/random_ops.py(91): random_normal
  /home/yslee/.local/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py(1176): op_dispatch_handler
  /home/yslee/.local/lib/python3.8/site-packages/tensorflow/python/util/traceback_utils.py(150): error_handler
  /tmp/ipykernel_237025/1858185163.py(55): <module>
  /home/yslee/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py(3508): run_code
  /home/yslee/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py(3448): run_ast_nodes
  /home/yslee/.local/l

In [32]:
multihead_attention.trainable_variables[1]

<tf.Variable 'multihead_attention_16/dense_64/bias:0' shape=(256,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 

In [44]:
multihead_attention.count_params()

263168